<a href="https://colab.research.google.com/github/nalpata/proyecto_aplicado_preservantes/blob/main/notebooks/Proyecto_1_Hito_2-PresentacionFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG Baseline - Proyecto Aplicado: Preservantes

En este notebook construimos el baseline de un sistema RAG usando un conjunto de PDFs
sobre preservantes. Incluye:

1. Carga e ingesta de PDFs
2. Preprocesamiento básico y chunking
3. Generación de embeddings
4. Creación de un vector store
5. Retriever (similarity search)
6. Benchmark (Precision@k sobre un set de preguntas)


In [ ]:
## Instalación de librerías (celda de código)
!pip install -q langchain langchain-community langchain-text-splitters \
               chromadb sentence-transformers pypdf


In [ ]:
##Importaciones y configuración básica
import os
from pathlib import Path

# LangChain imports
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# Para evaluación básica
from typing import List, Dict
import numpy as np

# Para ver resultados
from pprint import pprint


In [ ]:
# RESETEAR TODO PARA PARTIR LIMPIO EN COLAB

import os, shutil

# 1) Ir a /content
%cd /content

# 2) Borrar cualquier clone previo duplicado
if os.path.exists("proyecto_aplicado_preservantes"):
    shutil.rmtree("proyecto_aplicado_preservantes")
    print("🗑️ Carpeta borrada: proyecto_aplicado_preservantes")

# 3) Clonar de nuevo desde tu GitHub
!git clone https://github.com/nalpata/proyecto_aplicado_preservantes.git

# 4) Entrar a la carpeta correcta
%cd proyecto_aplicado_preservantes

print("\n🎉 Listo. Ahora estamos en el repo correcto sin duplicados.")
!ls


In [ ]:
# Ruta base del proyecto en Colab
BASE_PATH = Path("/content/proyecto_aplicado_preservantes")

DATA_PDF_DIR = BASE_PATH / "data" / "pdfs"          # aquí PDFs de preservantes
CHROMA_DIR   = BASE_PATH / "chroma_preservantes"   # carpeta donde se guardará el vector store

BASE_PATH.mkdir(parents=True, exist_ok=True)
CHROMA_DIR.mkdir(parents=True, exist_ok=True)

print("Base path:", BASE_PATH)
print("PDF dir:", DATA_PDF_DIR)
print("Chroma dir:", CHROMA_DIR)


In [ ]:
##Carga de documentos (ingesta de PDFs)
def load_pdfs(pdf_dir: Path):
    """
    Carga todos los PDFs de una carpeta usando LangChain.
    Devuelve una lista de Documents.
    """
    loader = DirectoryLoader(
        str(pdf_dir),
        glob="*.pdf",
        loader_cls=PyPDFLoader,
        show_progress=True
    )
    docs = loader.load()
    return docs

raw_docs = load_pdfs(DATA_PDF_DIR)
len(raw_docs), raw_docs[0]


In [ ]:
##Preprocesamiento
def clean_metadata(docs):
    """
    Normaliza  los metadatos: agrega un campo 'source'
    y mantiene solo lo relevante.
    """
    cleaned = []
    for d in docs:
        meta = d.metadata or {}
        source = meta.get("source", "")
        # Nos quedamos con un metadata simple
        new_meta = {
            "source": source,
            "page": meta.get("page", None)
        }
        d.metadata = new_meta
        cleaned.append(d)
    return cleaned

docs = clean_metadata(raw_docs)
len(docs), docs[0].metadata


**Chunking baseline**

Usamos RecursiveCharacterTextSplitter con:
- chunk_size = 800
- chunk_overlap = 200



In [ ]:
##Chunking
##Usamos RecursiveCharacterTextSplitter como baseline.

CHUNK_SIZE = 800
CHUNK_OVERLAP = 200

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
)

chunks = text_splitter.split_documents(docs)
len(chunks), chunks[0]


In [ ]:
##cuántos PDFs y de qué archivo vienen los chunks
from collections import Counter

print("N° de documentos originales:", len(raw_docs))
print("Fuentes (PDFs) originales:")
for src in sorted({d.metadata.get("source") for d in raw_docs}):
    print(" -", src)

print("\nN° de chunks:", len(chunks))
print("N° de chunks por PDF:")
conteo = Counter(d.metadata.get("source") for d in chunks)
for src, c in conteo.items():
    print(f"{src}: {c}")


Usamos chunking gerarquico porque los chunks no estan balanceados y se genera un pdf dominante

In [ ]:
## Uso chunking gerarquico
from langchain_text_splitters import RecursiveCharacterTextSplitter
from uuid import uuid4

# Splitter de nivel alto (bloques grandes)
high_level_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    length_function=len,
)

# Splitter de nivel bajo (para el vector store)
low_level_splitter = RecursiveCharacterTextSplitter(
    chunk_size=700,
    chunk_overlap=150,
    length_function=len,
)

def hierarchical_chunk(docs):
    """
    1. Divide en bloques grandes (nivel 1)
    2. Cada bloque grande se subdivide en chunks pequeños (nivel 2)
    3. Añade metadatos de jerarquía (parent_id, level1_index)
    """
    level1_docs = high_level_splitter.split_documents(docs)

    final_chunks = []
    for idx, d in enumerate(level1_docs):
        parent_id = str(uuid4())  # id único del bloque grande

        # subdividir este bloque
        sub_docs = low_level_splitter.split_documents([d])

        for s in sub_docs:
            meta = dict(s.metadata)
            meta["parent_id"] = parent_id
            meta["level1_index"] = idx
            s.metadata = meta
            final_chunks.append(s)

    return final_chunks

hier_chunks = hierarchical_chunk(docs)
len(hier_chunks), hier_chunks[0].metadata


In [ ]:
from pathlib import Path
from langchain_community.vectorstores import Chroma

BASE_PATH = Path("/content/proyecto_aplicado_preservantes")
CHROMA_HIER_DIR = BASE_PATH / "chroma_preservantes_hier"

import shutil
shutil.rmtree(CHROMA_HIER_DIR, ignore_errors=True)
CHROMA_HIER_DIR.mkdir(parents=True, exist_ok=True)

vector_store_hier = Chroma.from_documents(
    documents=hier_chunks,
    embedding=embeddings,                  # el mismo modelo multilingüe
    persist_directory=str(CHROMA_HIER_DIR)
)

vector_store_hier.persist()
print("Vector store HIER creado con", vector_store_hier._collection.count(), "documentos")


In [ ]:
retriever_hier = vector_store_hier.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 20}
)


In [ ]:
def inspeccionar_query_con(retriever, query: str, k: int = 5):
    docs = retriever.invoke(query)[:k]
    print("Query:", query, "\n")
    for i, d in enumerate(docs, 1):
        print(f"--- Documento {i} ---")
        print("Source:", d.metadata.get("source"), "| Page:", d.metadata.get("page"),
              "| level1_index:", d.metadata.get("level1_index"))
        print(d.page_content[:500], "...\n")

inspeccionar_query_con(retriever_hier, "¿What are the antimicrobial effects of sodium benzoate, sodium nitrite, and potassium sorbate?")


**Modelo de embeddings**

In [ ]:
EMBEDDING_MODEL_NAME = "sentence-transformers/distiluse-base-multilingual-cased-v2"

embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME
)

print("Modelo cargado:", EMBEDDING_MODEL_NAME)


**Vector Store**

In [ ]:
CHROMA_DIR.mkdir(parents=True, exist_ok=True)
#Cuando uso el jerárquico
CHROMA_HIER_DIR.mkdir(parents=True, exist_ok=True)


In [ ]:
vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=str(CHROMA_DIR)
)

vector_store.persist()
print("Vector store creado con", vector_store._collection.count(), "documentos")


In [ ]:
from langchain_community.vectorstores import Chroma

vector_store_hier = Chroma.from_documents(
    documents=hier_chunks,    # chunks jerárquicos
    embedding=embeddings      # mismo modelo multilingüe
)

print("Vector store jerárquico creado en memoria con:",
      vector_store_hier._collection.count(), "chunks")


**Retriever jerarquico**

In [ ]:
# Retriever jerárquico
retriever_hier = vector_store_hier.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 20}
)

query_ejemplo = "¿Qué es un preservante y qué función cumple en alimentos?"
resultados_hier = retriever_hier.invoke(query_ejemplo)

len(resultados_hier), resultados_hier[0]


In [ ]:
for i, d in enumerate(resultados_hier, 1):
    print(f"\n### Documento {i} ###")
    print("Source:", d.metadata.get("source"), "| Page:", d.metadata.get("page"))
    print(d.page_content[:300], "...\n")


In [ ]:
def evaluate_retriever(retriever, eval_queries, k=5, nombre="Evaluación"):
    print(f"\n=== Evaluando retriever: {nombre} ===\n")

    scores = []

    for item in eval_queries:
        query = item["query"]
        keywords = item["relevant_keywords"]

        # Recuperar documentos
        docs = retriever.invoke(query)[:k]

        # Precision@k manual
        hits = 0
        for doc in docs:
            text = doc.page_content.lower()
            if any(kw.lower() in text for kw in keywords):
                hits += 1

        precision = hits / k
        scores.append(precision)

        print(f"Query: {query}")
        print(f"Precision@{k}: {precision:.2f}\n")

    print(f"Precision@{k} promedio: {sum(scores)/len(scores):.2f}")
    return scores


**Naive RAG**

In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)


In [ ]:
##Baseline: solo mostrar textos recuperados
def show_retrieval(query: str, k: int = 5, retriever=retriever):
    # Con LangChain nuevo el retriever se invoca así:
    docs = retriever.invoke(query)
    docs = docs[:k]

    print(f"Query: {query}\n")
    for i, d in enumerate(docs, start=1):
        print(f"--- Documento {i} ---")
        print("Source:", d.metadata.get("source"), "Page:", d.metadata.get("page"))
        print(d.page_content[:500], "...")
        print()

# Prueba
show_retrieval("Tipos de preservantes utilizados en bebidas", retriever=retriever_hier)


**Integramos un LLM para responder**

In [ ]:
!pip install -q langchain-openai langchain-community openai tiktoken


In [ ]:
!pip install -q langchain langchain-openai langchain-community langchain-text-splitters
!pip install -q langchain-core
!pip install -q langchain-experimental
!pip install -q langchainhub
!pip install -q lc-retrieval


In [ ]:
import os
import getpass

from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough


In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Ingresa tu OPENAI_API_KEY: ")


In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)


In [ ]:
def ask_rag(query: str, k: int = 5, retriever=retriever_hier):
    # 1. Recuperar documentos relevantes
    docs = retriever.invoke(query)
    docs = docs[:k]

    # 2. Construir el contexto a partir de los chunks
    context = "\n\n---\n\n".join(d.page_content for d in docs)

    # 3. Armar el prompt para el LLM
    prompt = f"""
Eres un asistente experto en preservantes de alimentos.
Responde usando EXCLUSIVAMENTE la información del contexto.

Contexto:
{context}

Pregunta: {query}

Respuesta en español, clara y concisa:
"""

    # 4. Llamar al modelo
    response = llm.invoke(prompt)

    # 5. Mostrar resultado y fuentes
    print("Pregunta:", query)
    print("\n Respuesta:\n")
    print(response.content)

    print("\n Fuentes:")
    for d in docs:
        print("-", d.metadata.get("source"), "| page", d.metadata.get("page"))


In [ ]:
ask_rag("Tipos de preservantes utilizados en bebidas", retriever=retriever_hier)


**Benchmark  (Precision@k)**

In [ ]:
retriever_hier = vector_store_hier.as_retriever(
    search_type="mmr",          # búsqueda diversificada
    search_kwargs={
        "k": 5,                 # número final de documentos que regresará
        "fetch_k": 20           # número de documentos que explora primero
    }
)


In [ ]:
eval_queries = [
{
  "query": "¿Qué es un preservante antimicrobiano?",
  "relevant_keywords": [
    "preservante antimicrobiano",
    "conservante antimicrobiano",
    "inhibición microbiana",
    "inhibe el crecimiento microbiano",
    "sustancia antimicrobiana",
    "agente antimicrobiano",
    "inhibición de microorganismos",

    "antimicrobial preservative",
    "antimicrobial agent",
    "microbial growth inhibition",
    "inhibits microbial growth"
  ]
},
{
  "query": "¿Cuáles son los factores que afectan la efectividad de los preservantes?",
  "relevant_keywords": [
    "efectividad de los preservantes",
    "factores que afectan la efectividad",
    "actividad de agua",
    "aw",
    "concentración del conservante",
    "concentración inhibitoria",
    "pKa del conservante",
    "interacción con composición del alimento",

    "preservative effectiveness",
    "factors influencing preservative efficacy",
    "water activity",
    "aw value",
    "preservative concentration",
    "food composition interaction",
    "minimum inhibitory concentration"
  ]
},
{
  "query": "¿Qué se entiende por vida útil de un alimento?",
  "relevant_keywords": [
    "vida útil del alimento",
    "vida útil",
    "deterioro microbiano",
    "estabilidad del alimento",
    "seguridad alimentaria",
    "calidad durante el almacenamiento",

    "shelf life",
    "food shelf life",
    "food spoilage",
    "microbial spoilage",
    "quality stability",
    "storage stability"
  ]
}
]


In [ ]:
scores_hier = evaluate_retriever(
    retriever_hier,
    eval_queries,
    k=5,
    nombre="Jerárquico (MMR + chunking estructural)"
)


In [ ]:
from typing import List
import numpy as np

def precision_at_k(query: str, retrieved_docs: List, keywords: List[str], k: int = 5):
    """
    Calcula Precision@k verificando si los documentos recuperados contienen keywords relevantes.
    """
    hits = 0
    for doc in retrieved_docs[:k]:
        text = doc.page_content.lower()
        # Si alguna keyword aparece en el texto => HIT
        if any(keyword.lower() in text for keyword in keywords):
            hits += 1

    return hits / k  # Precision@k


def evaluate_retriever_precision(retriever, eval_queries, k: int = 5, nombre: str = "Modelo"):
    """
    Aplica Precision@k a un conjunto de queries y muestra resultados.
    """
    print(f"\n=== Evaluando retriever: {nombre} ===\n")

    scores = []
    for item in eval_queries:
        query = item["query"]
        keywords = item["relevant_keywords"]

        # Recuperar documentos
        retrieved = retriever.invoke(query)

        # Calcular Prec@k
        score = precision_at_k(query, retrieved, keywords, k)
        scores.append(score)

        print(f"Query: {query}")
        print(f"Precision@{k}: {score:.2f}\n")

    print(f"Precision@{k} promedio: {np.mean(scores):.2f}")
    return scores


In [ ]:
scores_hier = evaluate_retriever_precision(
    retriever_hier,
    eval_queries,
    k=5,
    nombre="Jerárquico (MMR)"
)


**CONCLUSIONES**

Corpus heterogéneo (inglés/español): Los documentos contienen conceptos relevantes en distintos idiomas, lo que afecta la recuperación cuando la evaluación depende de keywords únicamente en español o traducciones exactas.

Evaluación basada en coincidencia de palabras clave: Precision@k penaliza documentos que son relevantes conceptualmente, pero no contienen literalmente las palabras clave definidas.

Preguntas conceptuales difíciles: Consultas de tipo “¿Qué es…?” requieren definiciones explícitas que pueden no aparecer como tal en el corpus o estar formuladas con vocabulario técnico, reduciendo la recuperación efectiva.

Tamaño y calidad del corpus: Aunque el corpus es valioso, varias fuentes no están estructuradas pedagógicamente y contienen tablas, fórmulas o párrafos extensos, lo que dificulta la segmentación óptima.

**POSIBLES MEJORAS PARA SIGUENTE HITO**

Mejorar los embeddings. Adoptar un modelo más robusto y científico multilingüe

Optimizar el proceso de chunking: Usar chunking híbrido (estructura + semántica + tamaño).

Incluir metadatos explícitos (subtítulos, figuras, secciones) para mejorar contexto jerárquico.

Mejorar la evaluación: Expandir keywords con sinónimos y variaciones técnicas.

